In [12]:
import pandas as pd
import numpy as np
import random as rd
from tqdm import tqdm

class Enviroment:
    def __init__(self):
        self.env = np.array([[0,0,0,0],
                            [0,-1,0,-1],
                            [0,0,0,-1],
                            [-1,0,0,2]])
        self.end = False
        self.shape = self.env.shape
        self.current_state = (0,0)
        self.reward = 0

    def terminate_state(self):
        row,col = self.current_state
        if self.env[row][col] == 2:
            self.end = True
            self.reward = 1

        elif self.env[row][col] == -1:
            self.end = True
            self.reward = -1

        else:
            self.end = False
            self.reward = 0
            
    def step(self,action):
        row,col = self.current_state
        if action == 2:
            if col+1 < self.shape[1]:
                self.current_state = (row,col+1)
        if action == 0:
            if col-1 >= 0:
                self.current_state = (row,col-1)
        if action == 1:
            if row-1 >= 0:
                self.current_state = (row-1,col)
        if action == 3:
            if row+1 < self.shape[0]:
                self.current_state = (row+1,col)

        self.terminate_state()
        return self.current_state , self.end , self.reward
    
    def reset(self):
        self.env = np.array([[0,0,0,0],
                    [0,-1,0,-1],
                    [0,0,0,-1],
                    [-1,0,0,2]])
        self.end = False
        self.shape = self.env.shape
        self.current_state = (0,0)
        self.reward = 0


    def render(self):
        row,col = self.env.shape
        self.update_env = self.env.copy()
        env_rend = ''
        for i in range(row):
            for j in range(col):
                state = self.env[i][j]
                if self.current_state == (i,j):
                    self.update_env[i][j] = 1
                    env_rend += 'S'
                elif state == 0:
                    env_rend += 'F'
                elif state == -1:
                    env_rend += 'H'
                else:
                    env_rend += 'G'
            env_rend += '\n'
        print(env_rend)

In [13]:
env = Enviroment()
env.step(2)
env.render()

FSFF
FHFH
FFFH
HFFG



In [14]:
env.step(2)
env.render()

FFSF
FHFH
FFFH
HFFG



In [15]:
env.reset()

In [18]:
def generate_numcod(rows, cols):
    numcod = {}
    num = 0
    for i in range(rows):
        for j in range(cols):
            numcod[num] = (i, j)
            num += 1
    return numcod

# Generate the numcod dictionary
numcod = generate_numcod(4, 4)
print(numcod)

{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (1, 0), 5: (1, 1), 6: (1, 2), 7: (1, 3), 8: (2, 0), 9: (2, 1), 10: (2, 2), 11: (2, 3), 12: (3, 0), 13: (3, 1), 14: (3, 2), 15: (3, 3)}


In [19]:
def findPossibleActions(matrix):
    m = len(matrix)
    n = len(matrix[0])
    action_map = {}
    for i in range(m):
        for j in range(n):
            if matrix[i][j] in [-1,2]:
                continue
            directions = []
            if j-1 > -1 and matrix[i][j-1] != -1:
                directions.append(0)
            if j+1 < n and matrix[i][j+1] != -1:
                directions.append(2)
            if i-1 > -1 and matrix[i-1][j] != -1:
                directions.append(1)
            if i+1 < m and matrix[i+1][j] != -1:
                directions.append(3)
            action_map[i*4+j] = directions
    return action_map

In [20]:
# Define the grid environment
menv = np.array([[0, 0, 0, 0],
                [0, -1, 0, -1],
                [0, 0, 0, -1],
                [-1, 0, 0, 2]])

# Define the action map
action_map = {0: 'left',
              1: 'down',
              2: 'right',
              3: 'up'}
posact = findPossibleActions(menv)

In [46]:
posact

{0: [2, 3],
 1: [0, 2],
 2: [0, 2, 3],
 3: [0],
 4: [1, 3],
 6: [1, 3],
 8: [2, 1],
 9: [0, 2, 3],
 10: [0, 1, 3],
 13: [2, 1],
 14: [0, 2, 1]}

In [49]:
def setRMatrix(rmatrix,loss,win):
    reward = 10
    regret = -10
    for l in loss:
        rmatrix[l[0]][l[1]] = regret
    for w in win:
        rmatrix[w[0]][w[1]] = reward
    return rmatrix
        

In [90]:
rmatrix = np.zeros((16,4))
qmatrix = np.zeros((16,4))
# L D R U
# 0 1 2 3
loss = [(1,3),(3,3),(4,2),(6,0),(6,2),(8,3),(9,1),(10,2),(13,0)]
win = [(14,2)]

rmatrix = setRMatrix(rmatrix,loss,win)

In [91]:
def max_indices_random(lst):
    max_val = max(lst)
    max_indices = [i for i, val in enumerate(lst) if val == max_val]
    return np.random.choice(max_indices)


y = 0.9
done = False
count = 0
for i in tqdm(range(1000)):
    state1 = 0
    done = False
    maze = Enviroment()
    while done != True:
        ran = rd.random()
        action = rd.choice(posact[state1])
        obs,done,reward = maze.step(action)
        state2 = (obs[0]*4)+obs[1]
        qmatrix[state1][action] = rmatrix[state1][action] + y * max([qmatrix[state2][a] for a in list(range(4))])  
        state1 = state2        

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1856.81it/s]


In [92]:
rmatrix

array([[  0.,   0.,   0.,   0.],
       [  0.,   0.,   0., -10.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0., -10.],
       [  0.,   0., -10.,   0.],
       [  0.,   0.,   0.,   0.],
       [-10.,   0., -10.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0., -10.],
       [  0., -10.,   0.,   0.],
       [  0.,   0., -10.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [-10.,   0.,   0.,   0.],
       [  0.,   0.,  10.,   0.],
       [  0.,   0.,   0.,   0.]])

In [93]:
qmatrix

array([[ 0.     ,  0.     ,  5.9049 ,  5.9049 ],
       [ 5.31441,  0.     ,  6.561  ,  0.     ],
       [ 5.9049 ,  0.     ,  5.9049 ,  7.29   ],
       [ 6.561  ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  5.31441,  0.     ,  6.561  ],
       [ 0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  6.561  ,  0.     ,  8.1    ],
       [ 0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  5.9049 ,  7.29   ,  0.     ],
       [ 6.561  ,  0.     ,  8.1    ,  8.1    ],
       [ 7.29   ,  7.29   ,  0.     ,  9.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  7.29   ,  9.     ,  0.     ],
       [ 8.1    ,  8.1    , 10.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ]])

In [94]:
y = 0.9
done = False
count = 0
state1 = 0
env = Enviroment()
while done != True:
    env.render()
    action = max_indices_random(qmatrix[state1])
    obs,done,reward = env.step(action)
    state2 = (obs[0]*4)+obs[1]
    qmatrix[state1][action] = rmatrix[state1][action] + y * max([qmatrix[state2][a] for a in list(range(4))])  
    state1 = state2        

SFFF
FHFH
FFFH
HFFG

FFFF
SHFH
FFFH
HFFG

FFFF
FHFH
SFFH
HFFG

FFFF
FHFH
FSFH
HFFG

FFFF
FHFH
FFSH
HFFG

FFFF
FHFH
FFFH
HFSG



In [95]:
env.env

array([[ 0,  0,  0,  0],
       [ 0, -1,  0, -1],
       [ 0,  0,  0, -1],
       [-1,  0,  0,  2]])